<a href="https://colab.research.google.com/github/Gionauta/Avanti-Bootcamp-Machine-Learning/blob/main/Codes/YOLO_NAS_S/AdamW_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💿 Install Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/Avanti_Bootcamp/avantiDataReduc.zip

In [ ]:
!ls

# 📦 Required Dependencies



In [ ]:
!pip install pyproject-toml

In [ ]:
!pip install super-gradients

In [ ]:

import os
import shutil
import random
from tqdm.notebook import tqdm
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val
import torch
from super_gradients.training import models
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback


# 💾 Datasets and DataLoaders

In [ ]:
dataset_params = {
    'data_dir':'/content/avantiDataReduc',
    'train_images_dir':'/content/avantiDataReduc/kaggle/input/face-detection-dataset/images/train',
    'train_labels_dir':'/content/avantiDataReduc/kaggle/input/face-detection-dataset/labels/train',
    'val_images_dir':'/content/avantiDataReduc/kaggle/input/face-detection-dataset/images/val',
    'val_labels_dir':'/content/avantiDataReduc/kaggle/input/face-detection-dataset/labels/val',
    'test_images_dir':'/content/avantiDataReduc/kaggle/input/face-detection-dataset/images/val',
    'test_labels_dir':'/content/avantiDataReduc/kaggle/input/face-detection-dataset/labels/val',
     'classes': ['face']
}


In [ ]:
MODEL_ARCH = 'yolo_nas_m'
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
MAX_EPOCHS = 35
CHECKPOINT_DIR = f'/content/drive/MyDrive/Avanti_Bootcamp/DATA_SET/RUNs'
EXPERIMENT_NAME = f'RUN_S_ADAM'


In [ ]:
train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 1
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 1
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 1
    }
)

In [ ]:
train_data.dataset.transforms

In [ ]:
train_data.dataset.plot()

# ⚙️ Initializing the model

In [ ]:
model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    pretrained_weights="coco"
)


# 🔍 Training parameters

In [ ]:
train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "LinearBatchLRWarmup",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "CosineLRScheduler",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "AdamW",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9999, "decay_type": "exp", "beta": 15},
    "max_epochs": MAX_EPOCHS,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
        ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.5,


            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

# ⏳ Training the model

In [ ]:
from super_gradients.training import Trainer

trainer = Trainer(experiment_name=EXPERIMENT_NAME, ckpt_root_dir=CHECKPOINT_DIR)

In [ ]:
trainer.train(model=model,
              training_params=train_params,
              train_loader=train_data,
              valid_loader=val_data)

# 📈 Inference and Visualization

In [ ]:
best_model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    checkpoint_path=f"average_model.pth"
).to(DEVICE)

In [ ]:
img_1 = ""
best_model.predict(img_1).show()

In [ ]:
img_2 = ""
best_model.predict(img_2).show()